In [1]:
import pandas as pd
import ast
import numpy as np
from pathlib import Path  
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import geopandas as gpd
import pandas as pd
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
from matplotlib.patches import Patch
from sklearn.decomposition import PCA, FactorAnalysis
import matplotlib.colors as mcolors

C:\Users\peiyaozhao\AppData\Local\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the Excel file
df  = pd.read_csv('../../data/raw/msa-municipality.csv')
agency_df  = pd.read_csv('../../data/tidy/transit-counties-vmt-manually-filled.csv')

agency_df.rename(columns={'county_name': 'County'}, inplace=True)
agency_df

,Agency,City,State,County,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,1596235000,0,0,0,0,0,0,335497633,0
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,0,0,14383040,0,0,0,0,0,0
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,0,449235,77895324,0,0,11633307,0,0,111558
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,0,0,2792754,0,0,0,0,0,0
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,0,0,424795,24447143,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1316,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,796799,0,0,0,0,0,0,72466,0
1317,Bay State Cruise Company,NaN,MA,NaN,155873,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1318,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,0,0,42733,0,0,0,0,0,0
1319,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,0,0,19489,0,0,0,0,0,0


In [3]:
# Create a new dataframe
data = []

# Iterate through each row in the original dataframe
for index, row in df.iterrows():
    msa = row.iloc[0]  # Get the MSA
    counties = ast.literal_eval(row.iloc[1])  # Convert the string representation of the list back to a list
    
    # Append each county with its corresponding MSA to the new data list
    for county in counties:
        data.append([county, msa])

# Create a new dataframe from the data list
msa_county = pd.DataFrame(data, columns=['County', 'MSA'])

# Separate county and state
msa_county [['County Name', 'State']] = msa_county ['County'].str.split(', ', expand=True)
msa_county  = msa_county[['County Name', 'State', 'MSA']]

# Remove the word "County" from the "County Name" column
msa_county['County Name'] = msa_county['County Name'].str.replace('County', '').str.strip()

# Ensure the 'county' column in df1 and the 'County Name' column in df2 are in the same format
# Remove the word "County" if present in 'county' column
#agency_df['county'] = emissions['county'].str.replace('County', '').str.strip()

# Merge the dataframes
merged_df = pd.merge(agency_df, msa_county, how='left', left_on=['County', 'State'], right_on=['County Name', 'State'])

# Drop the 'County Name' column as it's redundant after the merge
#merged_df = merged_df.drop(columns=['County Name'])
merged_df.to_csv('../../data/tidy/transit-counties-vmt-fuel-msa.csv', index=False)
merged_df

,Agency,City,State,County,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),...,Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles),County Name,MSA
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,...,0,0,0,0,0,0,335497633,0,Kings,"New York-Newark-Jersey City, NY-NJ-PA"
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,...,0,14383040,0,0,0,0,0,0,Kings,"New York-Newark-Jersey City, NY-NJ-PA"
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,...,449235,77895324,0,0,11633307,0,0,111558,Kings,"New York-Newark-Jersey City, NY-NJ-PA"
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,...,0,2792754,0,0,0,0,0,0,Kings,"New York-Newark-Jersey City, NY-NJ-PA"
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,...,0,424795,24447143,0,0,0,0,0,Kings,"New York-Newark-Jersey City, NY-NJ-PA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,...,0,0,0,0,0,0,72466,0,Milwaukee,"Milwaukee-Waukesha, WI"
1334,Bay State Cruise Company,NaN,MA,NaN,155873,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,NaN
1335,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,...,0,42733,0,0,0,0,0,0,Pierce,"Seattle-Tacoma-Bellevue, WA"
1336,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,...,0,19489,0,0,0,0,0,0,Providence,"Providence-Warwick, RI-MA"


In [4]:
# Drop the 'County Name' column as it's redundant after the merge
merged_df = merged_df.drop(columns=['County Name'])

In [5]:
merged_df

,Agency,City,State,County,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles),MSA
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,1596235000,0,0,0,0,0,0,335497633,0,"New York-Newark-Jersey City, NY-NJ-PA"
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,0,0,14383040,0,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,0,449235,77895324,0,0,11633307,0,0,111558,"New York-Newark-Jersey City, NY-NJ-PA"
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,0,0,2792754,0,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,0,0,424795,24447143,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,796799,0,0,0,0,0,0,72466,0,"Milwaukee-Waukesha, WI"
1334,Bay State Cruise Company,NaN,MA,NaN,155873,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1335,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,0,0,42733,0,0,0,0,0,0,"Seattle-Tacoma-Bellevue, WA"
1336,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,0,0,19489,0,0,0,0,0,0,"Providence-Warwick, RI-MA"


In [7]:
# Drop rows where 'MSA' is NaN or blank
merged_df = merged_df.dropna(subset=['MSA'])
merged_df

,Agency,City,State,County,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles),MSA
0,MTA New York City Transit,Brooklyn,NY,Kings,0,0,0,0,0,0,1596235000,0,0,0,0,0,0,335497633,0,"New York-Newark-Jersey City, NY-NJ-PA"
1,MTA New York City Transit,Brooklyn,NY,Kings,3290382,0,0,0,0,0,0,0,14383040,0,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
2,MTA New York City Transit,Brooklyn,NY,Kings,23223866,0,0,3857115,0,0,0,449235,77895324,0,0,11633307,0,0,111558,"New York-Newark-Jersey City, NY-NJ-PA"
3,MTA New York City Transit,Brooklyn,NY,Kings,1074143,0,0,0,0,0,0,0,2792754,0,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
4,MTA New York City Transit,Brooklyn,NY,Kings,146008,3522121,0,0,0,0,0,0,424795,24447143,0,0,0,0,0,"New York-Newark-Jersey City, NY-NJ-PA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,McKinney Avenue Transit Authority,Dallas,TX,Dallas,0,0,0,0,0,0,320120,0,0,0,0,0,0,66958,0,"Dallas-Fort Worth-Arlington, TX"
1333,City of Milwaukee,Milwaukee,WI,Milwaukee,0,0,0,0,0,0,796799,0,0,0,0,0,0,72466,0,"Milwaukee-Waukesha, WI"
1335,"County of Pierce, dba: Pierce County Ferry",Tacoma,WA,Pierce,184209,0,0,0,0,0,0,0,42733,0,0,0,0,0,0,"Seattle-Tacoma-Bellevue, WA"
1336,Rhode Island Department of Transportation,Providence,RI,Providence,121806,0,0,0,0,0,0,0,19489,0,0,0,0,0,0,"Providence-Warwick, RI-MA"


In [8]:
merged_df.columns

Index(['Agency', 'City', 'State', 'County', 'Diesel (gal)', 'Gasoline (gal)',
       'Liquefied Petroleum Gas (gal equivalent)',
       'Compressed Natural Gas (gal equivalent)', 'Bio-Diesel (gal)',
       'Other Fuel (gal/gal equivalent)', 'Electric Propulsion (kwh)',
       'Electric Battery (kwh)', 'Diesel (miles)', 'Gasoline (miles)',
       'Liquefied Petroleum Gas (miles)', 'Compressed Natural Gas (miles)',
       'Other Fuel (miles)', 'Electric Propulsion (miles)',
       'Electric Battery (miles)', 'MSA'],
      dtype='object')

In [9]:
# Group by 'MSA' and sum the relevant columns
msa_sums_transit = merged_df.groupby('MSA')[['Diesel (gal)', 'Gasoline (gal)',
       'Liquefied Petroleum Gas (gal equivalent)',
       'Compressed Natural Gas (gal equivalent)', 'Bio-Diesel (gal)',
       'Other Fuel (gal/gal equivalent)', 'Electric Propulsion (kwh)',
       'Electric Battery (kwh)', 'Diesel (miles)', 'Gasoline (miles)',
       'Liquefied Petroleum Gas (miles)', 'Compressed Natural Gas (miles)',
       'Other Fuel (miles)', 'Electric Propulsion (miles)',
       'Electric Battery (miles)']].sum().reset_index()
msa_sums_transit

,MSA,Diesel (gal),Gasoline (gal),Liquefied Petroleum Gas (gal equivalent),Compressed Natural Gas (gal equivalent),Bio-Diesel (gal),Other Fuel (gal/gal equivalent),Electric Propulsion (kwh),Electric Battery (kwh),Diesel (miles),Gasoline (miles),Liquefied Petroleum Gas (miles),Compressed Natural Gas (miles),Other Fuel (miles),Electric Propulsion (miles),Electric Battery (miles)
0,"Akron, OH",498438,89147,0,849308,0,0,0,0,2619706,560570,0,3166425,0,0,0
1,"Albany, GA",39176,1760,0,147318,0,0,0,0,148053,12499,0,439611,0,0,0
2,"Albany-Schenectady-Troy, NY",2042304,223610,0,0,0,0,0,42126,8632669,1343264,0,0,0,0,0
3,"Albuquerque, NM",1404572,381365,0,224908,792273,0,0,0,6161642,3268665,0,989182,0,0,0
4,"Allentown-Bethlehem-Easton, PA-NJ",677850,126382,0,475045,0,0,0,0,3865849,1128538,0,1603714,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,"Yakima, WA",178977,28063,0,0,0,0,0,0,730634,275527,0,0,0,0,0
254,"York-Hanover, PA",130184,907139,0,179394,0,0,0,0,515734,6299147,0,553503,0,0,0
255,"Youngstown-Warren-Boardman, OH-PA",287458,56240,0,0,0,0,0,0,1467533,486469,0,0,0,0,0
256,"Yuba City, CA",206466,32653,0,0,0,0,0,0,926082,0,0,0,0,0,0


In [11]:
msa_sums_transit.to_csv('../../data/tidy/msa-transit-vmt-fuel.csv', index=False)